# Day 2 Meta and Full Predictions of Pokemon Worlds 2025

If you have not seen it yet, please go checkout the first notebook, [worlds2025_deck_sim](https://github.com/dlf57/pkmnworlds25_deck_sim/blob/main/worlds2025_deck_sim.ipynb), and the day 1 notebook, [worlds2025_day1_sim](https://github.com/dlf57/pkmnworlds25_deck_sim/blob/main/worlds2025_day1_sim.ipynb)

### Day 2 Meta
The simulated Day 2 meta was not all that far off from the Day 1 prediction. I think the big surprises are Grimmsnarl falling out of the top 5 graphic and raging bolt getting a large boost. A major note to make, I am setting Zoroark and Blissey to 50% win in every matchup as I have no meta information at this time on these deck's performances. I did drop Crustle and Ethan's Typhlosion because I do not believe any made Day 2.

|Deck	|Day 2 Simulated Share| Day 2 Actual Share |
|--|--|--|
|gholdengo-ex|	32.4%| 29.9%|
|dragapult-dusknoir|	21.4%|16.5%|
|gardevoir-ex-sv|	12.6%|9.4%|
|grimmsnarl-froslass|	10.2%|<4.7%|
|raging-bolt-ogerpon|	6.9%|12.6%|


### Lets Get into it!
With that being the meta, let's rerun our simulations and see what comes out on top!

In [1]:
import pandas as pd
import numpy as np
import random

### Load Meta & Matchups

We are still going to be using the matchups from the top 16 online tournaments but now looking at the day 1 meta.

In [2]:
# Load csv files
matchups = pd.read_csv("matchups-top16.csv")
meta = pd.read_csv("meta-worlds-day2.csv")

# Create a meta dictionary for Monte Carlo to randomly choose a deck from
meta['share'] = meta['share'] / 100.0
meta_dict = dict(zip(meta['deck'], meta['share']))

### Setup our Win Probability Matrix P[Deck1,Deck2]

To build out the win probability matrix we cannot just take the `win_rate` as that would be looking at how likely the deck is to win just one singular match. Becasue the format is best of 3, we need to find the match win probability for this scenario. 

What are our possible win paths? (where $p$ is our win rate)
 1. WW  - $P_{\text{WW}} = p \times p = p^2$
 2. WLW - $P_{\text{WLW}} = p \times (1-p) \times p = p^2 (1-p)$
 3. LWW - $P_{\text{LWW}} = (1-p) \times p \times p = p^2 (1-p)$

Now we sum these scenarios to be able to calculate the probability of a match win $P_{\text{match win}}$:

$P_{\text{match win}} = P_{\text{WW}} + P_{\text{WLW}} + P_{\text{LWW}}$

$P_{\text{match win}} = p^2 + 2p^2(1-p)$

$P_{\text{match win}} = p^2 \big( 1 + 2(1-p) \big)$

$P_{\text{match win}} = p^2 \big( 3 - 2p \big)$

In scenarios where decks have not faced eachother, we are going to make the assumption that the match will be 50/50. While not realistic, it is the best we can do with the information at this time

**CAVEAT** - Ties are possible in the TCG. For the purpose of this simulation we will be treating ties as a loss. For Worlds, this is not that detrimental as there are set numbers of wins needed to get through Day 1 and it is very unlikely to win out the tournament with multiple ties.

In [3]:
# Get list of all decks
decks = sorted(set(meta['deck']))
deck_index = {d:i for i,d in enumerate(decks)}
n = len(decks)

unique_decks = set(matchups["deck1"]).union(set(matchups["deck2"]))
filtered_matchups = matchups[matchups["deck1"].isin(deck_index) & matchups["deck2"].isin(deck_index)]

# Build win probability matrix P[a,b] = P(a beats b)
P = np.zeros((n,n))

for _, row in filtered_matchups.iterrows():
    a = deck_index[row['deck1']]
    b = deck_index[row['deck2']]

    # If total is NaN or zero, treat as no data
    total = row['total'] if not pd.isna(row['total']) else 0
    wins = row['wins'] if not pd.isna(row['wins']) else 0

    if total == 0:
        # No games → fall back to 50%
        p_match = 0.5
    else:
        # Calculate the match win probability
        p_win = wins / total
        p_match = p_win**2 * (3 - 2*p_win)

    P[a,b] = p_match

# mirror matches will always be 50%
for i in range(n):
    P[i,i] = 0.5

### Day 2, Top 16, & Champion

Since Day 1 is already done, we can just jump right into Day 2, cut and Champion simulations!

In [4]:
def simulate_day2(deck_name, meta_dist, swiss_rounds=4, cut_wins=3, sims=100000):
    top16_count = 0
    win_event_count = 0
    decks_list, fracs = zip(*meta_dist.items())
    fracs = np.array(fracs) / sum(fracs)

    for _ in range(sims):
        wins = 0
        for _ in range(swiss_rounds):
            opp = random.choices(decks_list, weights=fracs, k=1)[0]
            pwin = P[deck_index[deck_name], deck_index[opp]]
            if random.random() < pwin:
                wins += 1

        made_cut = wins >= cut_wins
        if made_cut:
            top16_count += 1
            # Simulate Top 16 (4 rounds)
            for _ in range(4):
                opp = random.choices(decks_list, weights=fracs, k=1)[0]
                pwin = P[deck_index[deck_name], deck_index[opp]]
                if random.random() >= pwin:
                    break
            else:
                win_event_count += 1

    return top16_count / sims, win_event_count / sims


results = []
for d in decks:
    p_cut, p_win = simulate_day2(d, meta_dict)
    r = {}
    r.update({'Deck': d})
    r.update({'Top 16%': p_cut})
    r.update({'Win %': p_win})
    results.append(r)

df_results = pd.DataFrame(results)
df_results['Top 16%'] = df_results['Top 16%'] * 100
df_results['Win %'] = df_results['Win %'] * 100
df_results.sort_values(by=['Win %'], ascending=False)

,Deck,Top 16%,Win %
1,charizard-dusknoir,36.081,2.738
9,grimmsnarl-froslass,32.301,2.209
3,dragapult-charizard,28.896,1.573
6,flareon-noctowl,27.943,1.505
8,gholdengo-ex,27.535,1.398
11,joltik-box,25.956,1.236
5,dragapult-ex,24.311,1.032
4,dragapult-dusknoir,23.926,0.991
12,raging-bolt-ogerpon,23.633,0.944
7,gardevoir-ex-sv,20.803,0.691


### Tournament Outcome

We still see Charizard/Dusknoir being a top deck choice for winning Worlds. Surprisingly Grimmsnarl is still rated high, though I think this is an artifact of the online tournaments. It seems that Worlds Competitors came highly prepared for the matchup. 

I am tempted to join the Charizard builds together as Ojvind is running a build with both Dusknoir and Pidgeot ex. As it heavily relies on Dusknoir for damange and getting behind in prizes for counter catcher and powering up Charizard, I am still maintaining that it is a Charizard/Dusknoir deck as most Charizard/Pidgeot decks forgo the crucial Dusknoir line. 

It will be interesting to see who the finalists are that make it to Championship Sunday! (will be updating this tonight with the results and my prediction on which deck will win!)